# Implementación base

1. Entrenar un modelo QDA sobre el dataset *iris* utilizando las distribuciones *a priori* a continuación ¿Se observan diferencias?¿Por qué cree? _Pista: comparar con las distribuciones del dataset completo, **sin splitear**_.
    1. Uniforme (cada clase tiene probabilidad 1/3)
    2. Una clase con probabilidad 0.9, las demás 0.05 (probar las 3 combinaciones)

In [3]:
import numpy as np
from numpy.linalg import det, inv

In [97]:
class ClassEncoder:
  def fit(self, y):
    self.names = np.unique(y)
    self.name_to_class = {name:idx for idx, name in enumerate(self.names)}
    self.fmt = y.dtype
    # Q1: por que no hace falta definir un class_to_name para el mapeo inverso?

  def _map_reshape(self, f, arr):
    return np.array([f(elem) for elem in arr.flatten()]).reshape(arr.shape)
    # Q2: por que hace falta un reshape?
    # A2: para que el return de _map_reshape devuelva un array con las mismas dimensiones
    #     que el el input (arr). Esto le da consistencia e interoperabilidad al metodo.

  def transform(self, y):
    return self._map_reshape(lambda name: self.name_to_class[name], y)

  def fit_transform(self, y):
    self.fit(y)
    return self.transform(y)

  def detransform(self, y_hat):
    return self._map_reshape(lambda idx: self.names[idx], y_hat)

In [98]:
class BaseBayesianClassifier:
  def __init__(self):
    self.encoder = ClassEncoder()

  def _estimate_a_priori(self, y):
    a_priori = np.bincount(y.flatten().astype(int)) / y.size
    # Q3: para que sirve bincount?
    return np.log(a_priori)

  def _fit_params(self, X, y):
    # estimate all needed parameters for given model
    raise NotImplementedError()

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    raise NotImplementedError()

  def fit(self, X, y, a_priori=None):
    # first encode the classes
    y = self.encoder.fit_transform(y)

    # if it's needed, estimate a priori probabilities
    self.log_a_priori = self._estimate_a_priori(y) if a_priori is None else np.log(a_priori)

    # check that a_priori has the correct number of classes
    assert len(self.log_a_priori) == len(self.encoder.names), "A priori probabilities do not match number of classes"

    # now that everything else is in place, estimate all needed parameters for given model
    self._fit_params(X, y)
    # Q4: por que el _fit_params va al final? no se puede mover a, por ejemplo, antes de la priori?
    return self.encoder.name_to_class

  def predict(self, X):
    # this is actually an individual prediction encased in a for-loop
    m_obs = X.shape[1]
    y_hat = np.empty(m_obs, dtype=self.encoder.fmt)

    for i in range(m_obs):
      encoded_y_hat_i = self._predict_one(X[:,i].reshape(-1,1))
      y_hat[i] = self.encoder.names[encoded_y_hat_i]

    # return prediction as a row vector (matching y)
    return y_hat.reshape(1,-1)

  def _predict_one(self, x):
    # calculate all log posteriori probabilities (actually, +C)
    log_posteriori = [ log_a_priori_i + self._predict_log_conditional(x, idx) for idx, log_a_priori_i
                  in enumerate(self.log_a_priori) ]

    # return the class that has maximum a posteriori probability
    return np.argmax(log_posteriori)

In [99]:
class QDA(BaseBayesianClassifier):

  def _fit_params(self, X, y):
    # estimate each covariance matrix
    self.inv_covs = [inv(np.cov(X[:,y.flatten()==idx], bias=True))
                      for idx in range(len(self.log_a_priori))]
    # Q5: por que hace falta el flatten y no se puede directamente X[:,y==idx]?
    # Q6: por que se usa bias=True en vez del default bias=False?
    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]
    # Q7: que hace axis=1? por que no axis=0?

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    inv_cov = self.inv_covs[class_idx]
    unbiased_x =  x - self.means[class_idx]
    return 0.5*np.log(det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x

### 1. Seteo de datos - Iris

In [100]:
# hiperparámetros
rng_seed = 6543

In [101]:
from sklearn.datasets import load_iris

def get_iris_dataset():
  data = load_iris()
  X_full = data.data
  y_full = np.array([data.target_names[y] for y in data.target.reshape(-1,1)])
  return X_full, y_full

X_full_iris, y_full_iris = get_iris_dataset()

print(f"X: {X_full_iris.shape}, Y:{y_full_iris.shape}")

X: (150, 4), Y:(150, 1)


In [102]:
def transpose(X, y):
    # transpose so observations are column vectors
    return X.T, y.T
    
def accuracy(y_true, y_pred):
  return (y_true == y_pred).mean()

X_full_iris, y_full_iris = transpose(X_full_iris, y_full_iris)
print(f"X: {X_full_iris.shape}, Y:{y_full_iris.shape}")

X: (4, 150), Y:(1, 150)


### 1. A) Uniforme

___Como indica la consigna, vamos a trabajar sin splitear.___

In [81]:
#Entrenamos un QDA y medimos su accuracy

qda = QDA()
names_class = qda.fit(X_full_iris, y_full_iris, a_priori=np.array([1/3, 1/3, 1/3]))
print(names_class)
train_acc = accuracy(y_full_iris, qda.predict(X_full_iris))
print(f"Train (apparent) error for iris with Uniform prior is {1-train_acc:.4f}")

{'setosa': 0, 'versicolor': 1, 'virginica': 2}
Train (apparent) error for iris with Uniform likelihood is 0.0200


### 1. B1) 0.9 0.05 0.05

In [83]:
#Entrenamos un QDA y medimos su accuracy

qda = QDA()
qda.fit(X_full_iris, y_full_iris, a_priori=np.array([0.9, 0.05, 0.05]))
train_acc = accuracy(y_full_iris, qda.predict(X_full_iris))
print(f"Train (apparent) error for iris with B1 prior is {1-train_acc:.4f}")

Train (apparent) error for iris with B1 likelihood is 0.0200


### 1. B2) 0.05 0.9 0.05

In [18]:
#Entrenamos un QDA y medimos su accuracy

qda = QDA()
qda.fit(X_full_iris, y_full_iris, a_priori=np.array([0.05, 0.9, 0.05]))

train_acc = accuracy(y_full_iris, qda.predict(X_full_iris))
print(f"Train (apparent) error for iris with B2 prior is {1-train_acc:.4f}")

Train (apparent) error for iris with B2 likelihood is 0.0333


### 1. B3) 0.05 0.05 0.9 

In [19]:
#Entrenamos un QDA y medimos su accuracy

qda = QDA()
qda.fit(X_full_iris, y_full_iris, a_priori=np.array([0.05, 0.05, 0.9]))

train_acc = accuracy(y_full_iris, qda.predict(X_full_iris))
print(f"Train (apparent) error for iris with B3 prior is {1-train_acc:.4f}")

Train (apparent) error for iris with B3 likelihood is 0.0400


In [25]:
# vemos la distribucion de las clases de iris
unique_values, counts = np.unique(y_full, return_counts=True)
tuple(zip(unique_values, counts))

(('setosa', 50), ('versicolor', 50), ('virginica', 50))

(RTA)  
Comparando las corridas, se observan variaciones en los resultados del error aparente de training en los casos B2 y B3. En el caso B1, el error es identico al que se da en A (cuando asumimos una distribución uniforme). 
El caso A responde a la máxima verosimilitud, ya que el dataset posee esa distribución (50/150 para cada clase). Sin embargo, coincide el error aparente con el caso B1. Lo cual es notorio ya que este último caso asume una distribución a priori del 90% para la clase setosa.

--- 

2. Repetir el punto anterior para el dataset penguin.

___Procediendo con el dataset penguins___

In [113]:
from sklearn.datasets import fetch_openml

def get_penguins():
    # get data
    df, tgt = fetch_openml(name="penguins", return_X_y=True, as_frame=True, parser='auto')

    # drop non-numeric columns
    df.drop(columns=["island","sex"], inplace=True)

    # drop rows with missing values
    mask = df.isna().sum(axis=1) == 0
    df = df[mask]
    tgt = tgt[mask]

    return df.values, tgt.to_numpy().reshape(-1,1)

In [114]:
X_full_peng, y_full_peng = get_penguins()
print(f"X: {X_full_peng.shape}, Y:{y_full_peng.shape}")

X: (342, 4), Y:(342, 1)


In [55]:
# vemos la distribución de las clases de penguins
unique_values, counts = np.unique(y_full_peng, return_counts=True)
tuple(zip(unique_values, counts, (str(round(count/y_full_peng.shape[1],4))+' %' for count in counts)))
#como son 3 clases, podemos reutilizar las distribuciones del caso anterior.

(('Adelie', 151, '0.4415 %'),
 ('Chinstrap', 68, '0.1988 %'),
 ('Gentoo', 123, '0.3596 %'))

En este caso la distribución no es uniforme, por lo que debería aumentar el error al elegir la distribución a priori "A".

In [115]:
# Volvemos a transponer para tenes las observaciones en vectores fila (a lo largo de las columnas)
X_full_peng, y_full_peng = transpose(X_full_peng, y_full_peng)
print(f"X: {X_full_peng.shape}, Y:{y_full_peng.shape}")

X: (4, 342), Y:(1, 342)


### 2. A)

In [84]:
#Entrenamos un QDA y medimos su accuracy
qda = QDA()
names_class = qda.fit(X_full_peng, y_full_peng, a_priori=np.array([1/3, 1/3, 1/3]))
print(names_class)
train_acc = accuracy(y_full_peng, qda.predict(X_full_peng))
print(f"Train (apparent) error is {1-train_acc:.4f}")

{'Adelie': 0, 'Chinstrap': 1, 'Gentoo': 2}
Train (apparent) error is 0.0088


### 2. B1)

In [85]:
#Entrenamos un QDA y medimos su accuracy
qda = QDA()
qda.fit(X_full_peng, y_full_peng, a_priori=np.array([0.9, 0.05, 0.05]))
train_acc = accuracy(y_full_peng, qda.predict(X_full_peng))
print(f"Train (apparent) error is {1-train_acc:.4f}")

Train (apparent) error is 0.0175


### 2. B2)

In [86]:
#Entrenamos un QDA y medimos su accuracy
qda = QDA()
qda.fit(X_full_peng, y_full_peng, a_priori=np.array([0.05, 0.9, 0.05]))
train_acc = accuracy(y_full_peng, qda.predict(X_full_peng))
print(f"Train (apparent) error is {1-train_acc:.4f}")

Train (apparent) error is 0.0351


### 2. B3)

In [89]:
#Entrenamos un QDA y medimos su accuracy
qda = QDA()
qda.fit(X_full_peng, y_full_peng, a_priori=np.array([0.05, 0.05, 0.9]))
train_acc = accuracy(y_full_peng, qda.predict(X_full_peng))
print(f"Train (apparent) error is {1-train_acc:.4f}")

Train (apparent) error is 0.0088


(RTA)  
Los modelos que convergen al error aparente más bajo son el A (suponiendo distribución a priori uniforme) y B3 (suponiendo una distribución del 90% de probabilidad para la clase "Gentoo", la cual tiene una frecuencia relativa del 35,96% en el dataset).   
Por otro lado, la distribución de probabilidad del caso B1 (90% para la clase Adeline, la cual tiene un 44,15% de frecuencia relativa en el dataset) dió un error mayor a los dos casos mencionados previamente. Esto se puede explicar porque la distribución a priori condiciona la predicción a la probabilidad que aparezca esa clase y, por las características del dataset, el modelo falla en las otras dos clases ya que discrimina peor entre sus atributos.

---

3. Implementar el modelo LDA, entrenarlo y testearlo contra los mismos sets que QDA (no múltiples prioris) ¿Se observan diferencias? ¿Podría decirse que alguno de los dos es notoriamente mejor que el otro?

_Por la consigna, se entinede que no debemos realizar un split del dataset, para mantener los mismos set que se usaron en QDA. A continuación se desarrolla la implementación de LDA para los set de Iris y Penguins con una distribución a priori de tipo Uniforme._

In [103]:
# Se hereda la clase QDA modificando la predicción para una matriz de covarianza constante a lo largo de las clases

class LDA(QDA):
    
    def _fit_params(self, X, y):
    # estimate ONE covariance matrix
    self.inv_covs = inv(np.cov(X, bias=True))
    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]
    
    def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    inv_cov = self.inv_covs[class_idx]
    #unbiased_x =  x - self.means[class_idx]
    unbiased_x =  x - 0.5 * self.means[class_idx]
      
    #return 0.5*np.log(det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x
    return self.means[class_idx].T @ inv_cov @ unbiased_x

### LDA Iris

In [112]:
#Entrenamos un LDA y medimos su accuracy
lda = LDA()
names_class = lda.fit(X_full_iris, y_full_iris, a_priori=np.array([1/3, 1/3, 1/3]))
print(names_class)
train_acc = accuracy(y_full_iris, lda.predict(X_full_iris))
print(f"Train (apparent) error for iris with Uniform prior is {1-train_acc:.4f}")

{'setosa': 0, 'versicolor': 1, 'virginica': 2}
Train (apparent) error for iris with Uniform prior is 0.6667


### LDA Penguins

In [116]:
#Entrenamos un LDA y medimos su accuracy
lda = LDA()
names_class = lda.fit(X_full_peng, y_full_peng, a_priori=np.array([1/3, 1/3, 1/3]))
print(names_class)
train_acc = accuracy(y_full_peng, lda.predict(X_full_peng))
print(f"Train (apparent) error for iris with Uniform prior is {1-train_acc:.4f}")

{'Adelie': 0, 'Chinstrap': 1, 'Gentoo': 2}
Train (apparent) error for iris with Uniform prior is 0.5585


(RTA)  
Los resultados de LDA sobre los mismo sets que se uso en QDA, para una distribución a priori uniforme, fueron ampliamente peores.  
El error aparente sobre el set de entrenamiento creció en más de un orden de magnitud:  
 - 33 veces mayor para el dataset de Iris (0.6667/0.02)
 - 63 veces mayor para el dataset de Penguins (0.5585/0.0088)

El hecho de asumir que las matrices de covarianza son iguales para todas las clases puede explicar el mal ajuste de los modelos LDA.  
Por otro lado, el modelo de Análisis Discriminante Cuadrático (QDA) no lleva a cabo esta suposición y permite que cada clase tenga su propia matriz de covarianza. Por lo tanto puede capturar mejor la complejidad de la estructura de los datos cuando las clases tienen diferentes varianzas.

---

4 Utilizar otros 2 (dos) valores de random seed para obtener distintos splits de train y test, y repetir la comparación del punto anterior ¿Las conclusiones previas se mantienen?

In [126]:
# hiperparámetros
rng_seeds = [125,99]

### Split

In [127]:
# preparing data, train - test validation
# 70-30 split
from sklearn.model_selection import train_test_split

def split_transpose(X, y, test_sz, random_state):
    # split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

    # transpose so observations are column vectors
    return X_train.T, y_train.T, X_test.T, y_test.T

def accuracy(y_true, y_pred):
  return (y_true == y_pred).mean()

### LDA

In [139]:
for i,rng_seed in enumerate(rng_seeds):
    print("Seed: ",rng_seed)
    print("Iiris")
    train_x, train_y, test_x, test_y = split_transpose(X_full_iris.T, y_full_iris.T, 0.3, rng_seed)
    
    lda = LDA()
    lda.fit(train_x, train_y, a_priori=np.array([1/3, 1/3, 1/3]))
    
    train_acc = accuracy(train_y, lda.predict(train_x))
    test_acc = accuracy(test_y, lda.predict(test_x))
    print(f"Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")
    
    print("-------")

    print("Penguins")
    train_x, train_y, test_x, test_y = split_transpose(X_full_peng.T, y_full_peng.T, 0.3, rng_seed)
    
    lda = LDA()
    lda.fit(train_x, train_y, a_priori=np.array([1/3, 1/3, 1/3]))
    
    train_acc = accuracy(train_y, lda.predict(train_x))
    test_acc = accuracy(test_y, lda.predict(test_x))
    print(f"Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")
    print("-------")



Seed:  125
Iiris
Train (apparent) error is 0.6762 while test error is 0.6444
-------
Penguins
Train (apparent) error is 0.6318 while test error is 0.6602
-------
Seed:  99
Iiris
Train (apparent) error is 0.6381 while test error is 0.7333
-------
Penguins
Train (apparent) error is 0.6192 while test error is 0.6893
-------


### QDA

In [141]:
for i,rng_seed in enumerate(rng_seeds):
    print("Seed: ",rng_seed)
    print("Iiris")
    train_x, train_y, test_x, test_y = split_transpose(X_full_iris.T, y_full_iris.T, 0.3, rng_seed)
    
    lda = QDA()
    lda.fit(train_x, train_y, a_priori=np.array([1/3, 1/3, 1/3]))
    
    train_acc = accuracy(train_y, lda.predict(train_x))
    test_acc = accuracy(test_y, lda.predict(test_x))
    print(f"Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")
    
    print("-------")

    print("Penguins")
    train_x, train_y, test_x, test_y = split_transpose(X_full_peng.T, y_full_peng.T, 0.3, rng_seed)
    
    lda = LDA()
    lda.fit(train_x, train_y, a_priori=np.array([1/3, 1/3, 1/3]))
    
    train_acc = accuracy(train_y, lda.predict(train_x))
    test_acc = accuracy(test_y, lda.predict(test_x))
    print(f"Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")
    print("-------")



Seed:  125
Iiris
Train (apparent) error is 0.0286 while test error is 0.0000
-------
Penguins
Train (apparent) error is 0.6318 while test error is 0.6602
-------
Seed:  99
Iiris
Train (apparent) error is 0.0190 while test error is 0.0222
-------
Penguins
Train (apparent) error is 0.6192 while test error is 0.6893
-------


### Saving data in a DF

In [143]:
import pandas as pd

In [146]:
import pandas as pd

modelos = []
datasets = []
seeds = []
errores_train = []
errores_test = []

for rng_seed in rng_seeds:
    train_x, train_y, test_x, test_y = split_transpose(X_full_iris.T, y_full_iris.T, 0.3, rng_seed)
    lda = QDA()
    lda.fit(train_x, train_y, a_priori=np.array([1/3, 1/3, 1/3]))
    train_acc = accuracy(train_y, lda.predict(train_x))
    test_acc = accuracy(test_y, lda.predict(test_x))
    modelos.append('QDA')
    datasets.append('Iris')
    seeds.append(rng_seed)
    errores_train.append(1-train_acc)
    errores_test.append(1-test_acc)
    #------------
    train_x, train_y, test_x, test_y = split_transpose(X_full_peng.T, y_full_peng.T, 0.3, rng_seed)
    lda = QDA()
    lda.fit(train_x, train_y, a_priori=np.array([1/3, 1/3, 1/3]))
    train_acc = accuracy(train_y, lda.predict(train_x))
    test_acc = accuracy(test_y, lda.predict(test_x))
    modelos.append('QDA')
    datasets.append('Penguins')
    seeds.append(rng_seed)
    errores_train.append(1-train_acc)
    errores_test.append(1-test_acc)
    #------------
    #------------
    train_x, train_y, test_x, test_y = split_transpose(X_full_iris.T, y_full_iris.T, 0.3, rng_seed)
    lda = LDA()
    lda.fit(train_x, train_y, a_priori=np.array([1/3, 1/3, 1/3]))
    train_acc = accuracy(train_y, lda.predict(train_x))
    test_acc = accuracy(test_y, lda.predict(test_x))
    modelos.append('LDA')
    datasets.append('Iris')
    seeds.append(rng_seed)
    errores_train.append(1-train_acc)
    errores_test.append(1-test_acc)
    #------------
    train_x, train_y, test_x, test_y = split_transpose(X_full_peng.T, y_full_peng.T, 0.3, rng_seed)
    lda = LDA()
    lda.fit(train_x, train_y, a_priori=np.array([1/3, 1/3, 1/3]))
    train_acc = accuracy(train_y, lda.predict(train_x))
    test_acc = accuracy(test_y, lda.predict(test_x))
    modelos.append('LDA')
    datasets.append('Penguins')
    seeds.append(rng_seed)
    errores_train.append(1-train_acc)
    errores_test.append(1-test_acc)

# Crear el DataFrame
df = pd.DataFrame({
    'Modelo': modelos,
    'Dataset': datasets,
    'Seed': seeds,
    'Error (train)': errores_train,
    'Error (test)': errores_test
})

# Imprimir la tabla
df


,Modelo,Dataset,Seed,Error (train),Error (test)
0,QDA,Iris,125,0.028571,0.000000
1,QDA,Penguins,125,0.008368,0.009709
2,LDA,Iris,125,0.676190,0.644444
3,LDA,Penguins,125,0.631799,0.660194
4,QDA,Iris,99,0.019048,0.022222
5,QDA,Penguins,99,0.008368,0.009709
6,LDA,Iris,99,0.638095,0.733333
7,LDA,Penguins,99,0.619247,0.689320


(RTA)  
Podemos ver que el modelo QDA para Iris tiene un error en testing es de 0 para el seed 125. Esto se debe a que estamos usando una distribución a priori uniforme sobre un dataset uniforme con un estimador de discriminante cuadradático. La estimación por máxima verosimilitud logra ajustar los parámetros y predice correctamente el set de testing para ambos seeds.  
En cambio el modelo LDA no logra predecir correctamente en el set de testing obteniendo error superiores al 50%. Esto se debe a que la naturaleza lineal de su discriminante no le permite clasificar correctamente las clases. 

---

5. Estimar y comparar los tiempos de predicción de las clases `QDA` y `TensorizedQDA`. De haber diferencias ¿Cuáles pueden ser las causas?

In [148]:
class TensorizedQDA(QDA):

    def _fit_params(self, X, y):
        # ask plain QDA to fit params
        super()._fit_params(X,y)

        # stack onto new dimension
        self.tensor_inv_cov = np.stack(self.inv_covs)
        self.tensor_means = np.stack(self.means)

    def _predict_log_conditionals(self,x):
        unbiased_x = x - self.tensor_means
        inner_prod = unbiased_x.transpose(0,2,1) @ self.tensor_inv_cov @ unbiased_x

        return 0.5*np.log(det(self.tensor_inv_cov)) - 0.5 * inner_prod.flatten()

    def _predict_one(self, x):
        # return the class that has maximum a posteriori probability
        return np.argmax(self.log_a_priori + self._predict_log_conditionals(x))

In [154]:
%%timeit

model_qda = QDA()
model_qda.fit(train_x, train_y)
model_qda.predict(test_x)

12.7 ms ± 401 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [152]:
%%timeit

model_Tqda = TensorizedQDA()
model_Tqda.fit(train_x, train_y)
model_Tqda.predict(test_x)

5.37 ms ± 166 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


_midiendo solo la predicción_

In [156]:
model_qda = QDA()
model_qda.fit(train_x, train_y)

{'Adelie': 0, 'Chinstrap': 1, 'Gentoo': 2}

In [157]:
%%timeit
model_qda.predict(test_x)

11.8 ms ± 750 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [158]:
model_Tqda = TensorizedQDA()
model_Tqda.fit(train_x, train_y)

{'Adelie': 0, 'Chinstrap': 1, 'Gentoo': 2}

In [159]:
%%timeit
model_Tqda.predict(test_x)

4.61 ms ± 115 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [160]:
11.8/4.61 , 12.7/5.37

(2.5596529284164857, 2.364990689013035)

(RTA)  
Viendo los tiempos obtenidos en las 4 mediciones, se concluye que la diferencia reside en la predicción. Se puede ver que la clase que utiliza tensores para la predicción (cálculo de la probabilidad a posteriori) es más del doble de rápida que la clase QDA standard.  
Esto se explica comparando los métodos "_predict_log_conditionals":  
 - En QDA tenemos productos matriciales, los cuales se ven mediante el operador "@".
 - En TensorizedQDA tenemos operaciones element-wise que solamente utilizan operadores "+" y "*". Para poder hacer esto se utiliza el tensor "tensor_inv_cov" que contiene las matrices de covarianza inversas de todas las clases. También se usa el operador @, especificamente en "inner_prod", pero como las variables que computa son tensores, numpy realiza las operaciones elemento a elemento.

Si computaramos cantidad de iteraciones en ves del tiempo, la diferencia sería 3 a 1 para los sets utilizados. En este caso vemos una diferencia de 2,5 aproximadamente ya que la predicción para una clase demora levemente menos que para las tres en simultaneo. QDA realiza la probabilidad condicional para una sola observación y una sola clase a la vez mientras que TensorizedQDA realiza el cálculo para todas las clases simultáneamente.

---

---

# Optimización matemática

## QDA

Debido a la forma cuadrática de QDA, no se puede predecir para *n* observaciones en una sola pasada (utilizar $X \in \mathbb{R}^{p \times n}$ en vez de $x \in \mathbb{R}^p$) sin pasar por una matriz de *n x n* en donde se computan todas las interacciones entre observaciones. Se puede acceder al resultado recuperando sólo la diagonal de dicha matriz, pero resulta ineficiente en tiempo y (especialmente) en memoria. Aún así, es *posible* que el modelo funcione más rápido.

1. Implementar el modelo `FasterQDA` (se recomienda heredarlo de TensorizedQDA) de manera de eliminar el ciclo for en el método predict.
2. Comparar los tiempos de predicción de `FasterQDA` con `TensorizedQDA` y `QDA`.
3. Mostrar (puede ser con un print) dónde aparece la mencionada matriz de *n x n*, donde *n* es la cantidad de observaciones a predecir.
4.Demostrar que
$$
diag(A \cdot B) = \sum_{cols} A \odot B^T = np.sum(A \odot B^T, axis=1)
$$ es decir, que se puede "esquivar" la matriz de *n x n* usando matrices de *n x p*.
5.Utilizar la propiedad antes demostrada para reimplementar la predicción del modelo `FasterQDA` de forma eficiente. ¿Hay cambios en los tiempos de predicción?

## LDA

1. "Tensorizar" el modelo LDA y comparar sus tiempos de predicción con el modelo antes implementado. *Notar que, en modo tensorizado, se puede directamente precomputar $\mu^T \cdot \Sigma^{-1} \in \mathbb{R}^{k \times 1 \times p}$ y guardar eso en vez de $\Sigma^{-1}$.*
2. LDA no sufre del problema antes descrito de QDA debido a que no computa productos internos, por lo que no tiene un verdadero costo extra en memoria predecir "en batch". Implementar el modelo `FasterLDA` y comparar sus tiempos de predicción con las versiones anteriores de LDA.